In [1]:
import json
import pandas as pd
from pycaret.classification import setup, compare_models, save_model, pull, predict_model

# Carregar as configurações do arquivo config.json
try:
    with open('config.json', 'r') as f:
        config = json.load(f)
except FileNotFoundError:
    print("Arquivo config.json não encontrado.")
    raise

# Verificar a extensão do arquivo para carregar corretamente
dataset_path = f'./datasets/{config["file"]}'
try:
    if dataset_path.endswith('.csv'):
        df = pd.read_csv(dataset_path)
    elif dataset_path.endswith('.xlsx'):
        df = pd.read_excel(dataset_path)
    else:
        raise ValueError("Formato de arquivo não suportado. Use CSV ou XLSX.")
except Exception as e:
    print(f"Erro ao carregar o dataset: {e}")
    raise

# Verificar se o dataframe tem colunas e dados
if df.empty or len(df.columns) == 0:
    raise ValueError("Erro: O dataset está vazio ou sem colunas.")

# Verificar se a variável alvo está no dataset
if config["target"] not in df.columns:
    raise ValueError(f"A coluna alvo '{config['target']}' não está presente no dataset.")

# Verificar se a coluna de target contém valores válidos
if df[config["target"]].isnull().sum() > 0:
    raise ValueError(f"A coluna alvo '{config['target']}' contém valores nulos. Por favor, limpe ou preencha esses valores.")

# Verificações e ajustes de configuração
normalize = config.get("normalize", False)
normalize_method = config.get("normalization_method", None) if normalize else None

remove_multicollinearity = config.get("remove_multicollinearity", False)
multicollinearity_threshold = config.get("multicollinearity_threshold", None) if remove_multicollinearity else None

fold_strategy = config.get("fold_strategy", None)  # Se não for informado, usa None
fold_number = config.get("fold_number", 10) if fold_strategy else None  # Se fold_strategy não estiver marcado, ignoramos

# Setup do PyCaret
clf = setup(
    data=df,
    target=config["target"],
    session_id=config["session_id"],
    normalize=normalize,
    normalize_method=normalize_method,
    train_size=1 - config["test_size"],
    fold_strategy=fold_strategy,
    fold=fold_number,
    remove_multicollinearity=remove_multicollinearity,
    multicollinearity_threshold=multicollinearity_threshold
)

# Treinar os modelos e salvar o melhor
best_model = compare_models()

# Verificar se best_model foi retornado corretamente
if best_model is None or (isinstance(best_model, list) and len(best_model) == 0):
    raise ValueError("Nenhum modelo foi comparado ou retornado. Verifique as configurações.")

# Se best_model for uma lista, pegar o primeiro modelo
if isinstance(best_model, list):
    best_model = best_model[0]

# Salvar o melhor modelo
save_model(best_model, './models/best_model')

# Puxar os resultados e salvar em CSV
results = pull()
results.to_csv('./models/results.csv', index=False)

# Predizer no dataset e salvar as predicoes
predictions = predict_model(best_model)
predictions.to_csv('./models/predictions.csv', index=False)

# Salvar os parâmetros do melhor modelo
with open('./models/best_model_params.json', 'w') as f:
    json.dump(best_model.get_params(), f)

# Salvar o nome do modelo no arquivo config.json
with open('config.json', 'r') as f:
    config = json.load(f)

config['model_name'] = best_model.__class__.__name__  # Obtendo o nome da classe do modelo

with open('config.json', 'w') as f:
    json.dump(config, f)

print("Treinamento e predições completos. Resultados foram salvos.")


,Description,Value
0,Session id,1245
1,Target,Grupo
2,Target type,Binary
3,Original data shape,"(120, 45)"
4,Transformed data shape,"(120, 11)"
5,Transformed train set shape,"(84, 11)"
6,Transformed test set shape,"(36, 11)"
7,Numeric features,44
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0080
rf,Random Forest Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0270
ada,Ada Boost Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0070
gbc,Gradient Boosting Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0120
et,Extra Trees Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0230
lightgbm,Light Gradient Boosting Machine,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0200
ridge,Ridge Classifier,0.9764,0.9938,0.9550,1.0000,0.9746,0.9530,0.9575,0.0080
qda,Quadratic Discriminant Analysis,0.9653,0.9612,0.9350,1.0000,0.9607,0.9321,0.9407,0.0080
knn,K Neighbors Classifier,0.9639,0.9975,0.9500,0.9800,0.9603,0.9280,0.9349,0.0140
lda,Linear Discriminant Analysis,0.9639,1.0000,0.9300,1.0000,0.9603,0.9280,0.9349,0.0070


Transformation Pipeline and Model Successfully Saved


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Decision Tree Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


Treinamento e predições completos. Resultados foram salvos.


In [2]:
pip install pycaret

Note: you may need to restart the kernel to use updated packages.
